In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from constants import numeric_features, categorical_features
from sklearn.linear_model import LogisticRegression
from part2.shared import load_processed_data
from part2.shared import load_train_with_validation_data


In [ ]:
NUMBER_OF_EPOCHS = 250

In [ ]:

df = load_processed_data()
df = df[df["Target"].isin(["Graduate", "Dropout"])]
df["y"] = (df["Target"] == "Graduate").astype(int)
df = df.drop(columns=["Target", "Target encoded"], errors='ignore')

In [ ]:
X = df.drop(columns=["y"])
y = df["y"].values

In [ ]:
num_features = [c for c in numeric_features if c in X.columns]
cat_features = [c for c in categorical_features if c in X.columns]

In [ ]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

In [ ]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [ ]:
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])

In [ ]:
X_train_raw, X_val_raw, X_test_raw, y_train, y_val, y_test = load_train_with_validation_data(X, y)
X_train = full_pipeline.fit_transform(X_train_raw)
X_test = full_pipeline.transform(X_test_raw)
X_val = full_pipeline.transform(X_val_raw)

In [ ]:
from Logistic import CustomLogisticRegression
from sklearn.metrics import classification_report
import numpy as np

baseline_clf = CustomLogisticRegression(
    lr=0.05,
    epochs=NUMBER_OF_EPOCHS,
    batch_size=256,
)
baseline_clf.fit(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test
)

y_pred = baseline_clf.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))


In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
print("Liczebność klas:", dict(zip(unique, counts)))

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=1)

X_train_os, y_train_os = smote.fit_resample(X_train, y_train)

unique_os, counts_os = np.unique(y_train_os, return_counts=True)
print("Liczebność klas (po SMOTE):", dict(zip(unique_os, counts_os)))


In [29]:
from Logistic import CustomLogisticRegression
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

model_os = CustomLogisticRegression(
    lr=0.05,
    epochs=NUMBER_OF_EPOCHS,
    batch_size=64,
    verbose=True,
)

model_os.fit(X_train_os, y_train_os, X_val, y_val, X_test, y_test)

y_pred_os = model_os.predict(X_test)
print(classification_report(y_test, y_pred_os, digits=4))


Epoch 200/250  train_loss=0.2219  val_loss=0.2108
Epoch 220/250  train_loss=0.2214  val_loss=0.2098
Epoch 240/250  train_loss=0.2210  val_loss=0.2108
              precision    recall  f1-score   support

           0     0.8873    0.8873    0.8873       284
           1     0.9278    0.9278    0.9278       443

    accuracy                         0.9120       727
   macro avg     0.9075    0.9075    0.9075       727
weighted avg     0.9120    0.9120    0.9120       727



In [30]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=1)

X_train_us, y_train_us = rus.fit_resample(X_train, y_train)

unique_us, counts_us = np.unique(y_train_us, return_counts=True)
print("Liczebność klas (po undersamplingu):", dict(zip(unique_us, counts_us)))


Liczebność klas (po undersamplingu): {np.int64(0): np.int64(990), np.int64(1): np.int64(990)}


In [31]:
model_us = CustomLogisticRegression(
    lr=0.05,
    epochs=NUMBER_OF_EPOCHS,
    batch_size=64,
    verbose=True,
)

model_us.fit(X_train_us, y_train_us, X_val, y_val, X_test, y_test)

y_pred_us = model_us.predict(X_test)
print(classification_report(y_test, y_pred_us, digits=4))



Epoch   1/250  train_loss=0.4557  val_loss=0.4357
Epoch  20/250  train_loss=0.2787  val_loss=0.2635
Epoch  40/250  train_loss=0.2588  val_loss=0.2465
Epoch  60/250  train_loss=0.2500  val_loss=0.2367
Epoch  80/250  train_loss=0.2450  val_loss=0.2305
Epoch 100/250  train_loss=0.2418  val_loss=0.2270
Epoch 120/250  train_loss=0.2395  val_loss=0.2238
Epoch 140/250  train_loss=0.2378  val_loss=0.2231
Epoch 160/250  train_loss=0.2365  val_loss=0.2217
Epoch 180/250  train_loss=0.2355  val_loss=0.2202
Epoch 200/250  train_loss=0.2347  val_loss=0.2191
Epoch 220/250  train_loss=0.2340  val_loss=0.2189
Epoch 240/250  train_loss=0.2334  val_loss=0.2192
              precision    recall  f1-score   support

           0     0.8828    0.9014    0.8920       284
           1     0.9359    0.9233    0.9295       443

    accuracy                         0.9147       727
   macro avg     0.9093    0.9123    0.9108       727
weighted avg     0.9152    0.9147    0.9149       727



In [ ]:
from sklearn.metrics import recall_score

results_summary = {
    "Model": ["Oryginalne dane", "SMOTE", "Undersampling"],
    "Accuracy": [
        accuracy_score(y_test, baseline_clf.predict(X_test)),
        accuracy_score(y_test, model_os.predict(X_test)),
        accuracy_score(y_test, model_us.predict(X_test))
    ],
    "Recall (klasa 0)": [
        recall_score(y_test, baseline_clf.predict(X_test), pos_label=0),
        recall_score(y_test, model_os.predict(X_test), pos_label=0),
        recall_score(y_test, model_us.predict(X_test), pos_label=0)
    ],
    "F1 (klasa 0)": [
        f1_score(y_test, baseline_clf.predict(X_test), pos_label=0),
        f1_score(y_test, model_os.predict(X_test), pos_label=0),
        f1_score(y_test, model_us.predict(X_test), pos_label=0)
    ]
}

import pandas as pd

summary_df = pd.DataFrame(results_summary)
print(summary_df)
